In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 최신 뉴스 기사

In [9]:
naver_news_df_fin = get_dataframe_from_bigquery('news', 'news_daily_0'+today, bigquerykey_path)

In [10]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [11]:
naver_news_df_final = naver_news_df_fin[(naver_news_df_fin['제목'].str.contains(need_kwd1) & 
                                        #   naver_news_df_fin['제목'].str.contains(need_kwd2)) &
                                          ~naver_news_df_fin['제목'].str.contains(except_kwd)) |
                                          naver_news_df_fin['제목'].str.contains(need_kwd3)].reset_index(drop=True)

In [12]:
save_dataframe_to_bigquery(naver_news_df_final, 'news', 'news_daily_listup', bigquerykey_path)

Data inserted into table news_daily_listup successfully.
